In [1]:
import pandas as pd
import numpy as np
import os
import datetime as dt
import calendar
from matplotlib import pyplot as plt
import scipy.stats as stats
import glob

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

from mpl_toolkits.mplot3d import Axes3D  #3Dplot
import seaborn as sns
import japanize_matplotlib


from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

from matplotlib.ticker import ScalarFormatter
import matplotlib.ticker as ptick ##これが必要！

In [2]:
files = glob.glob('../../../卒論関連書類/データ/king_history/king_history1/*')
version = len(files)-1
df_king =pd.read_csv(f'../../../卒論関連書類/データ/king_history/king_history1/king{version}.csv',converters={'DateTime':pd.to_datetime})



In [3]:
def Get_values(target,start_point,end_point):
    cols = ['DateTime', 'gap_price[intra-spot](円/kWh)','低下量合計', '停止・低下ユニット数合計','intra_volume（MWh/h）','spot_volume(kWh)', '太陽光絶対値誤差合算(実績-想定)(kWh)',
       '風力絶対値誤差合算(実績-想定)(kWh)']

    df_select = df_king[cols]

    re_col = ['DateTime', '価格差 (時間前市場 - スポット市場) (Yen/kWh)','出力低下量合計 (kW)', '停止・低下ユニット数合計 (基)','時間前市場取引量 (kWh)','スポット市場取引量 (kWh)', '太陽光誤差(実績-想定)合算 (kWh)',
        '風力誤差(実績-想定)合算値 (kWh)'
        #    , 'エリア毎の広域予備率の平均値(%)'
        ]
    df_select.columns = re_col
    df_select['時間前市場取引量 (kWh)'] =df_select['時間前市場取引量 (kWh)']*1000/2

    percent =50
    cols___=['DateTime', '価格差 (時間前市場 - スポット市場) (Yen/kWh)']+target
    df = df_select[cols___]
    df = df[(df['DateTime']>=start_point)&(df['DateTime']<=end_point)].dropna()


    df['judge'] =0

    df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'] =abs(df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'])
    judge_line = np.percentile(df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'],percent)

    df.loc[df['価格差 (時間前市場 - スポット市場) (Yen/kWh)']>judge_line,['judge']] =1
    df.drop(columns='価格差 (時間前市場 - スポット市場) (Yen/kWh)')
    return(df)


In [4]:
year=2022
start_point = dt.datetime(year,1,1,0,0)
end_point = dt.datetime(year,12,31,23,30)
target = ['出力低下量合計 (kW)','スポット市場取引量 (kWh)']
df_test =Get_values(target,start_point,end_point)

C:\Users\Annex Rakyu\AppData\Local\Temp\ipykernel_25576\2737269267.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['時間前市場取引量 (kWh)'] =df_select['時間前市場取引量 (kWh)']*1000/2


In [5]:
# 正則化している
# 標準化もしてる
# df_Y=df_test['judge']
df_X = df_test[target]
# 標準化
scaler = StandardScaler()
scaler.fit(np.array(df_X))
df_std = scaler.transform(np.array(df_X))
df_std = pd.DataFrame(df_std,columns=df_X.columns)

# 目的変数(Y)
# Y = np.array(df['gap_judge'])

# 説明変数(X)
# col_name = ['停止・低下ユニット数合計','spot_volume(kWh)']
X = np.array(df_std)
Y = np.array(df_test['judge'])
# データの分割(訓練データとテストデータ)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)




lr = LogisticRegression(penalty='l1',solver='liblinear') # ロジスティック回帰モデルのインスタンスを作成
lr.fit(X_train, Y_train) # ロジスティック回帰モデルの重みを学習

print("coefficient = ", lr.coef_)
print("intercept = ", lr.intercept_)


Y_pred = lr.predict(X_test)



print('confusion matrix = \n', confusion_matrix(y_true=Y_test, y_pred=Y_pred))
print('accuracy = ', accuracy_score(y_true=Y_test, y_pred=Y_pred))
print('precision = ', precision_score(y_true=Y_test, y_pred=Y_pred))
print('recall = ', recall_score(y_true=Y_test, y_pred=Y_pred))
print('f1 score = ', f1_score(y_true=Y_test, y_pred=Y_pred))



coefficient =  [[-0.1081072  0.2834366]]
intercept =  [0.01258483]
confusion matrix = 
 [[1445 1241]
 [1079 1491]]
accuracy =  0.558599695585997
precision =  0.5457540263543191
recall =  0.5801556420233464
f1 score =  0.5624292719728404


In [7]:
year=2022
start_point = dt.datetime(year,1,1,0,0)
end_point = dt.datetime(year,12,31,23,30)
target = ['出力低下量合計 (kW)','スポット市場取引量 (kWh)']
df_test =Get_values(target,start_point,end_point)

# 正則化している
# df_Y=df_test['judge']
df_X = df_test[target]
# 標準化
# scaler = StandardScaler()
# scaler.fit(np.array(df_X))
# df_std = scaler.transform(np.array(df_X))
# df_std = pd.DataFrame(df_std,columns=df_X.columns)

# 目的変数(Y)
# Y = np.array(df['gap_judge'])

# 説明変数(X)
# col_name = ['停止・低下ユニット数合計','spot_volume(kWh)']
X = np.array(df_X)
Y = np.array(df_test['judge'])
# データの分割(訓練データとテストデータ)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)




lr = LogisticRegression(penalty='l1',solver='liblinear',max_iter=1000) # ロジスティック回帰モデルのインスタンスを作成
lr.fit(X_train, Y_train) # ロジスティック回帰モデルの重みを学習

print("coefficient = ", lr.coef_)
print("intercept = ", lr.intercept_)


Y_pred = lr.predict(X_test)



print('confusion matrix = \n', confusion_matrix(y_true=Y_test, y_pred=Y_pred))
print('accuracy = ', accuracy_score(y_true=Y_test, y_pred=Y_pred))
print('precision = ', precision_score(y_true=Y_test, y_pred=Y_pred))
print('recall = ', recall_score(y_true=Y_test, y_pred=Y_pred))
print('f1 score = ', f1_score(y_true=Y_test, y_pred=Y_pred))

C:\Users\Annex Rakyu\AppData\Local\Temp\ipykernel_25576\2737269267.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['時間前市場取引量 (kWh)'] =df_select['時間前市場取引量 (kWh)']*1000/2


coefficient =  [[-9.40008910e-09  1.07687976e-07]]
intercept =  [-1.50352343]
confusion matrix = 
 [[1443 1243]
 [1075 1495]]
accuracy =  0.5589802130898022
precision =  0.5460189919649379
recall =  0.5817120622568094
f1 score =  0.5633006782215524


c:\Users\Annex Rakyu\.conda\envs\DS385\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [29]:
df

,DateTime,価格差 (時間前市場 - スポット市場) (Yen/kWh),風力誤差(実績-想定)合算値 (kWh),judge
66655,2022-01-01 00:00:00,5.11,24983.843486,1
66656,2022-01-01 00:30:00,5.93,27201.553453,1
66657,2022-01-01 01:00:00,5.71,30913.700497,1
66658,2022-01-01 01:30:00,3.27,27555.744709,1
66659,2022-01-01 02:00:00,3.54,24570.421317,1
...,...,...,...,...
79754,2022-09-30 21:30:00,1.92,42780.147236,0
79755,2022-09-30 22:00:00,0.13,36939.208319,0
79756,2022-09-30 22:30:00,0.57,42335.191793,0
79757,2022-09-30 23:00:00,0.29,38608.958776,0


In [10]:
year=2022
start_point = dt.datetime(year,1,1,0,0)
end_point = dt.datetime(year,12,31,23,30)
target = ['出力低下量合計 (kW)', '停止・低下ユニット数合計 (基)','時間前市場取引量 (kWh)','スポット市場取引量 (kWh)', '太陽光誤差(実績-想定)合算 (kWh)',
        '風力誤差(実績-想定)合算値 (kWh)']
df_test =Get_values(target,start_point,end_point)

# 正則化している
# df_Y=df_test['judge']
df_X = df_test[target]
# 標準化
# scaler = StandardScaler()
# scaler.fit(np.array(df_X))
# df_std = scaler.transform(np.array(df_X))
# df_std = pd.DataFrame(df_std,columns=df_X.columns)

# 目的変数(Y)
# Y = np.array(df['gap_judge'])

# 説明変数(X)
# col_name = ['停止・低下ユニット数合計','spot_volume(kWh)']
X = np.array(df_X)
Y = np.array(df_test['judge'])
# データの分割(訓練データとテストデータ)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)




lr = LogisticRegression(penalty='l1',solver='liblinear',max_iter=1000) # ロジスティック回帰モデルのインスタンスを作成
lr.fit(X_train, Y_train) # ロジスティック回帰モデルの重みを学習

print("coefficient = ", lr.coef_)
print("intercept = ", lr.intercept_)


Y_pred = lr.predict(X_test)



print('confusion matrix = \n', confusion_matrix(y_true=Y_test, y_pred=Y_pred))
print('accuracy = ', accuracy_score(y_true=Y_test, y_pred=Y_pred))
print('precision = ', precision_score(y_true=Y_test, y_pred=Y_pred))
print('recall = ', recall_score(y_true=Y_test, y_pred=Y_pred))
print('f1 score = ', f1_score(y_true=Y_test, y_pred=Y_pred))

C:\Users\Annex Rakyu\AppData\Local\Temp\ipykernel_25576\2737269267.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['時間前市場取引量 (kWh)'] =df_select['時間前市場取引量 (kWh)']*1000/2


coefficient =  [[ 1.02272403e-09 -2.78671738e-03  1.57328247e-06  1.00543478e-07
   4.90525217e-07 -5.57715000e-07]]
intercept =  [-2.14190954]
confusion matrix = 
 [[1215  748]
 [ 812 1157]]
accuracy =  0.6032553407934893
precision =  0.6073490813648293
recall =  0.5876079228034535
f1 score =  0.5973154362416107


c:\Users\Annex Rakyu\.conda\envs\DS385\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [23]:
# df_Y=df_test['judge']
df_X = df_test[target]
# 標準化
scaler = StandardScaler()
scaler.fit(np.array(df_X))
df_std = scaler.transform(np.array(df_X))
df_std = pd.DataFrame(df_std,columns=df_X.columns)

# 目的変数(Y)
# Y = np.array(df['gap_judge'])

# 説明変数(X)
# col_name = ['停止・低下ユニット数合計','spot_volume(kWh)']
X = np.array(df_std)
Y = np.array(df_test['judge'])
# データの分割(訓練データとテストデータ)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)




lr = LogisticRegression(penalty='l1', solver='liblinear') # ロジスティック回帰モデルのインスタンスを作成
lr.fit(X_train, Y_train) # ロジスティック回帰モデルの重みを学習

print("coefficient = ", lr.coef_)
print("intercept = ", lr.intercept_)


Y_pred = lr.predict(X_test)



print('confusion matrix = \n', confusion_matrix(y_true=Y_test, y_pred=Y_pred))
print('accuracy = ', accuracy_score(y_true=Y_test, y_pred=Y_pred))
print('precision = ', precision_score(y_true=Y_test, y_pred=Y_pred))
print('recall = ', recall_score(y_true=Y_test, y_pred=Y_pred))
print('f1 score = ', f1_score(y_true=Y_test, y_pred=Y_pred))

coefficient =  [[-0.10811531  0.28343473]]
intercept =  [0.01258498]
confusion matrix = 
 [[1445 1241]
 [1079 1491]]
accuracy =  0.558599695585997
precision =  0.5457540263543191
recall =  0.5801556420233464
f1 score =  0.5624292719728404


In [24]:
df_X

,出力低下量合計 (kW),スポット市場取引量 (kWh)
66655,29902740.0,16671250
66656,27860240.0,17250450
66657,27860240.0,17568350
66658,27860240.0,17867750
66659,27860240.0,17997900
...,...,...
84170,49386670.0,19267150
84171,50888670.0,19020650
84172,50888670.0,18906400
84173,50888670.0,18719700


In [4]:

cols = ['DateTime', 'gap_price[intra-spot](円/kWh)','低下量合計', '停止・低下ユニット数合計','intra_volume（MWh/h）','spot_volume(kWh)', '太陽光絶対値誤差合算(実績-想定)(kWh)',
       '風力絶対値誤差合算(実績-想定)(kWh)']

df_select = df_king[cols]

re_col = ['DateTime', '価格差 (時間前市場 - スポット市場) (Yen/kWh)','出力低下量合計 (kW)', '停止・低下ユニット数合計 (基)','時間前市場取引量 (kWh)','スポット市場取引量 (kWh)', '太陽光誤差(実績-想定)合算 (kWh)',
       '風力誤差(実績-想定)合算値 (kWh)'
    #    , 'エリア毎の広域予備率の平均値(%)'
       ]
df_select.columns = re_col
df_select['時間前市場取引量 (kWh)'] =df_select['時間前市場取引量 (kWh)']*1000/2

percent =50
# これは各年

# for year in range(2018,2023):

# for year in range(2018,2023):
year=2022
start_point = dt.datetime(year,1,1,0,0)
end_point = dt.datetime(year,12,31,23,30)

# まず、データがある分すべての相関をとる。
target =re_col[2:]
for name in target:
    df = df_select[['DateTime', '価格差 (時間前市場 - スポット市場) (Yen/kWh)',name]]
    df = df[(df['DateTime']>=start_point)&(df['DateTime']<=end_point)].dropna()


    df['judge'] =0

    df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'] =abs(df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'])
    judge_line = np.percentile(df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'],percent)

    df.loc[df['価格差 (時間前市場 - スポット市場) (Yen/kWh)']>judge_line,['judge']] =1

    value0 = df[df['judge']==0][name].values
    value1 = df[df['judge']==1][name].values
    times =df['DateTime'].dt.date.values



C:\Users\Annex Rakyu\AppData\Local\Temp\ipykernel_17056\2069483604.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['時間前市場取引量 (kWh)'] =df_select['時間前市場取引量 (kWh)']*1000/2
